## Capstone Project Notebook

Bowen Li

In [ ]:
!conda install -c conda-forge beautifulsoup4 --yes

!conda install -c conda-forge geopy --yes

!conda install -c conda-forge folium=0.5.0 --yes

In [ ]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import json
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans


To create the above dataframe:

In [ ]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

table = soup.find('table',class_ = 'wikitable sortable')

wiki_table_rows = table.find_all('tr')

data =[]

for row in wiki_table_rows:
     data.append([n.text.strip() for n in row.find_all('td')])
       

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  

print(df.head(5))

To create the above dataframe:

In [ ]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
df.head()

df["Neighbourhood"] = df.groupby("PostalCode")["Neighbourhood"].transform(lambda neigh: ', '.join(neigh))

df = df.drop_duplicates()


if(df.index.name != 'PostalCode'):
    df = df.set_index('PostalCode')

df['Neighbourhood'].replace("Not assigned", df["Borough"],inplace=True)
df.head()

In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [ ]:
df.shape

## Getting coordinates and add to the Toronto DataFrame

In [ ]:
!wget -q -O "toronto_coordinates.csv" http://cocl.us/Geospatial_data
print('Coordinates downloaded!')
coordinates = pd.read_csv('toronto_coordinates.csv')

In [ ]:
toronto_df_temp = df.set_index('PostalCode')
coordinates_temp = coordinates.set_index('Postal Code')
toronto_df_coordinates = pd.concat([toronto_df_temp, coordinates_temp], axis=1, join='inner')

toronto_df_coordinates.index.name = 'PostalCode'
toronto_df_coordinates.reset_index(inplace=True)

toronto_df_coordinates.head()